In [1]:
from utilities import *
from core.initial import Insertions
from docplex.cp.model import *

In [2]:
datafile = "D:/Google Drive/MSc MST-AUEB/_Thesis_/Main Thesis/Model Data.xlsx"
d = DataProvider(filepath=datafile, route='910')
model = CSPModel(d)
model.build_model()

model.data.head()

,initial_depot,final_depot,relief_point,time,trip_duration,start_time,end_time
trip,,,,,,,
0,Fix Station,Fix Station,NaN,05:00,40,300,340
1,Fix Station,Fix Station,NaN,05:20,40,320,360
2,Fix Station,Fix Station,NaN,05:55,40,355,395
3,Fix Station,Fix Station,NaN,06:20,44,380,424
4,Fix Station,Fix Station,NaN,06:40,44,400,444


In [ ]:
# initial = Insertions(model)
# initial.solve()

In [ ]:
# trips = [interval_var(start=trip.start_time,
#                       end=trip.end_time,
#                       size=trip.duration,
#                       name=f'Trip_{idx}') for idx, trip in enumerate(model.trips)]


In [3]:
ntrips = len(model.trips)
nduties = len(model.trips)

sub = CpoModel(name="Pricing_Subproblem")


min_start = model.data[start_time].min()
max_start = model.data[start_time].max()
min_end = model.data[end_time].min()
max_end = model.data[end_time].max()

duties = [interval_var(start=(min_start, max_start),
                       end=(min_end, max_end),
                       size=model.constraints.shift_span,
                       name=f"Duty_{i}",
                       optional=True)
          for i in range(nduties)]

trip2trip = integer_var_list(size=ntrips,
                             min=0,
                             max=ntrips + 1,
                             name='Trip2Trip')

trip2duty = integer_var_list(size=ntrips,
                             min=0,
                             max=nduties,
                             name='Trip2Duty')

start_time = [[integer_var(min=0,
                           max=model.constraints.shift_span,
                           name=f"StartTime-{i}-{j}")
               for j in range(nduties)] for i in range(ntrips)]

cdt = integer_var_list(size=nduties,
                       min=0,
                       max=model.constraints.continuous_driving,
                       name="CDT")

tdt = integer_var_list(size=nduties,
                       min=0,
                       max=model.constraints.total_driving,
                       name="TDT")

In [4]:
for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(
            trip2trip[i] == j, model.end_time_arr[i] <= model.start_time_arr[j]))

for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(
            trip2trip[i] == j, model.end_loc_arr[i] <= model.start_loc_arr[j]))


for i in range(ntrips):
    sub.add(trip2trip[i] != i)

for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(trip2trip[i] == j, trip2duty[i] == trip2duty[j]))

In [10]:
msol = sub.solve()

sduties = []
for i in range(ntrips):
    _out = f"{i:>2} -> {msol[trip2trip[i]]} | Duty: {msol[trip2duty[i]]}"
    print(_out)
    sduties.append(msol[trip2duty[i]])

print(set(sduties))

 ! --------------------------------------------------- CP Optimizer 20.1.0.0 --
 ! Satisfiability problem - 116 variables, 10150 constraints
 ! Initial process time : 0.06s (0.06s extraction + 0.00s propagation)
 !  . Log search space  : 600.2 (before), 600.2 (after)
 !  . Memory usage      : 2.5 MB (before), 2.5 MB (after)
 ! Using parallel search with 4 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
 *                     69  0.08s        1         4  = Trip2Duty_13
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 272
 ! Number of fails        : 0
 ! Total memory usage     : 16.0 MB (16.0 MB CP Optimizer + 0.0 MB Concert)
 ! Time spent in solve    : 0.08s (0.03s engine + 0.06s extraction)
 ! Search speed 

In [9]:
type(msol[trip2duty[0]])

int

In [14]:
trips = [8, 13, 30, 43, 46, 47, 49, 50, 52]

In [15]:
model.data.loc[trips]

,initial_depot,final_depot,relief_point,time,trip_duration,start_time,end_time
trip,,,,,,,
8,Fix Station,Fix Station,NaN,07:45,48,465,513
13,Fix Station,Fix Station,NaN,09:05,52,545,597
30,Fix Station,Fix Station,NaN,13:45,44,825,869
43,Fix Station,Fix Station,NaN,17:25,52,1045,1097
46,Fix Station,Fix Station,NaN,18:25,44,1105,1149
47,Fix Station,Fix Station,NaN,18:50,44,1130,1174
49,Fix Station,Fix Station,NaN,19:30,44,1170,1214
50,Fix Station,Fix Station,NaN,19:50,44,1190,1234
52,Fix Station,Fix Station,NaN,20:40,44,1240,1284


In [ ]:
for i in range(ntrips):
    sub.add(trip2trip[i] != 0)
    # sub.add(sum([trip2trip[j] == i for j in range(ntrips+1)]) == 1)

In [ ]:
for i in range(ntrips-1):
    for j in range(i + 1, ntrips):
        sub.add(end_before_start(trips[i], trips[j]))

In [ ]:
has_break = [[binary_var(name=f'Break_{i}_{j}') for j in range(nduties)] for i in range(ntrips)]
has_rest = [[binary_var(name=f'Rest_{i}_{j}') for j in range(nduties)] for i in range(ntrips)]

In [ ]:
trip_seq = sequence_var(trips)

In [ ]:
td_matrix = {}
for trip in model.trips:
    for duty in range(ntrips):
        td_matrix[(trip.ID, duty)] = binary_var(name=f"T{trip.ID}-D{duty}")

In [ ]:
end_of(trips[0]).equals(trips[1])

In [ ]:
INTERVAL_MAX